In [1]:
# !pip install --no-deps 'timm-0.1.26-py3-none-any.whl' > /dev/null
# !pip install omegaconf

In [11]:
!pip uninstall -y timm

Found existing installation: timm 0.1.26
Uninstalling timm-0.1.26:
  Successfully uninstalled timm-0.1.26


In [13]:
!pip install timm==0.3.2

  Using cached timm-0.3.2-py3-none-any.whl (244 kB)
  Attempting uninstall: timm
    Found existing installation: timm 0.4.9
    Uninstalling timm-0.4.9:
      Successfully uninstalled timm-0.4.9


In [7]:
!pip install --no-deps 'timm-0.1.26-py3-none-any.whl' > /dev/null

In [1]:
import sys
sys.path.insert(0, "timm-efficientdet-pytorch")
# sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "effdet")
import torch
import os
from datetime import datetime
import time
import random
import cv2
import pandas as pd
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from glob import glob
import pickle
from effdet_old import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet_old.efficientdet import HeadNet
from ensemble_boxes import *
SEED = 42
import gc
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [2]:
label_codes = {
"Pedestrian": 0,
"Biker": 1,
"Car": 2,
"Bus": 3,
"Skater": 4,
"Cart": 5
}
label_colors = {
0: (1,0,0),
1:(0,1,0),
2: (0,0,1),
 3: (1,1,0),
4:(0,1,1),
5:(1,1,1)
}

In [4]:
# with open('train_dataset.df', 'wb') as f:
#     pickle.dump(train_dataset.df, f)
# with open('train_dataset.df', 'rb') as f:
#     df = pickle.load(f)

In [3]:
transforms =  A.Compose([
#                 A.RandomSizedCrop(min_max_height=(800, 800), height=1024, width=1024, p=1),
            A.RandomCrop(512, 512),
#             A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)



In [4]:
class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = transforms(image=image)
            image = sample['image']
        return image,image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [5]:
DATA_ROOT_PATH = '/mnt/r4/aliev/stanford_voc/VOC2012/JPEGImages'
dataset = DatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=transforms
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    collate_fn=collate_fn
)

In [6]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d0')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 6
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()


net = load_net('effdet0_loss_055_state_dict.pt')

TypeError: 'FeatureInfo' object is not callable

In [7]:
images = torch.stack(images).cuda().float()
with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())

NameError: name 'images' is not defined

In [28]:
det[:,-1]

tensor([[1.5905e+02, 2.2716e+02, 1.4079e+01, 1.9785e+01, 3.7305e-02, 1.0000e+00],
        [1.2764e+01, 4.3111e+02, 2.2411e+01, 2.4791e+01, 3.5872e-02, 1.0000e+00]],
       device='cuda:0')

In [8]:
def make_predictions(images, score_threshold=0.12):
    images = torch.stack(images).cuda().float()
    predictions = []
    with torch.no_grad():
        det = net(images, torch.tensor([1]*images.shape[0]).float().cuda())
        for i in range(images.shape[0]):
            boxes = det[i].detach().cpu().numpy()[:,:4]    
            scores = det[i].detach().cpu().numpy()[:,4]
            labels = det[i].detach().cpu().numpy()[:,5]
            indexes = np.where(scores > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            predictions.append({
                'boxes': boxes[indexes],
                'scores': scores[indexes],
                'labels': labels[indexes]
            })
    return [predictions],det

def run_wbf(predictions, image_index, image_size=512, iou_thr=0.12, skip_box_thr=0.12, weights=None):
    boxes = [(prediction[image_index]['boxes']/(image_size-1)).tolist()  for prediction in predictions]
    scores = [prediction[image_index]['scores'].tolist()  for prediction in predictions]
    labels = [prediction[image_index]['labels'].tolist()  for prediction in predictions]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [55]:
images[0].shape

torch.Size([3, 512, 512])

In [ ]:
import matplotlib.pyplot as plt

for j, (images,image_ids) in enumerate(data_loader):
    break
predictions,det = make_predictions(images)

i = 0
sample = images[i].permute(1,2,0).cpu().numpy()

boxes, scores, labels = run_wbf(predictions, image_index=i)
boxes = boxes.astype(np.int32).clip(min=0, max=511)

fig, ax = plt.subplots(1, 1, figsize=(20, 16))

for box,label in zip(boxes,labels):
    cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), label_colors[label], 1)
    
ax.set_axis_off()
ax.imshow(sample)


In [ ]:
import matplotlib.pyplot as plt

for j, t in enumerate(val_loader):
    break

In [ ]:
with torch.no_grad():
    net.forward(t)

In [ ]:
train_dataset.image_ids()

In [ ]:
train_dataset.image_ids.index('coupa_video0_11128.jpg')

In [ ]:
fname = '/mnt/r4/aliev/stanford_voc/JPEGImages/bookstore_video1_4.jpg'
image = cv2.imread(fname, cv2.IMREAD_COLOR)


In [ ]:
plt.imshow(image)

In [ ]:
type(images[0])